In [1]:
import pandas as pd
import numpy as np
import scipy.signal as sig
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from imblearn.under_sampling import RandomUnderSampler

In [2]:
D = pd.read_csv("../assets/data/cleaned/D.csv", index_col=0)
E = pd.read_csv("../assets/data/cleaned/E.csv", index_col=0)

In [3]:
D.shape, E.shape

((1025, 24), (1380, 24))

In [4]:
def stat_summary(X, iqrl=0.25, iqrr=0.75):
    '''
    Return for each timestamp (hr) return min, max, 0.25 quantile, 0.75 quantile, mean, var
    '''
    return pd.DataFrame(np.array([np.min(X, axis=0), np.quantile(X, axis=0, q=iqrl), np.median(X, axis=0), 
          np.quantile(X, axis=0, q=iqrr), np.max(X, axis=0), np.mean(X, axis=0), np.var(X, axis=0), np.std(X, axis=0)]).transpose(),
                        columns=["Min", "0.25Q", "Med", "0.75Q", "Max", "Mean", "Var", "StD"])

def trend(X, span = None):
    '''
    Return EWMA trend for signal samples
    '''
    if not span:
        span = X.shape[1]
    return pd.DataFrame(X).apply(lambda x: x.ewm(span=span).mean(), axis=1)

def train_test_val_split(X, y=None, test_size=0.2,  val_size=0.4):
    from sklearn.model_selection import train_test_split
    if y.__len__() > 0:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
        X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=val_size)
        return X_train, X_test, X_val, y_train, y_test, y_val
    else:
        X_train, X_test = train_test_split(X, test_size=test_size)
        X_test, X_val = train_test_split(X_test, test_size=val_size)
        return X_train, X_test, X_val

def reshapeInput(X):
    X_ = []
    for x in X:
        X_.append(x.reshape(-1,1))
    return np.array(X_)

def shapeInput(X):
    X_ = []
    for x in X:
        X_.append(x.reshape(1,-1)[0])
    return np.array(X_)


def meanShift(X, window=None):
    '''
    Apply MA on convolved signal and compute rs means
    '''
    # At this point X must be EWMA convolved already
    if not window:
        window = X.shape[1] // 2
    return pd.DataFrame(X).apply(lambda x: x.rolling(window=window).mean(), axis=1).iloc[:,window-1:]

def gradient(X, shift=1, threshold=0):
    '''
    First order gradient of signal. Eq to diff for discrete signal
    '''
    return pd.DataFrame(X).apply(lambda x: x.diff(periods=shift), axis=1).iloc[:,threshold:]

def bucketPeaks(peakIndex):
    '''
    Bucket into first half and second half
    '''
    return(sum(peakIndex <= 12), sum(peakIndex > 12)) 

In [5]:
X = np.row_stack((D,E))
Y = [0]*(D.__len__()) + [1]*(E.__len__())

In [6]:
X_train, X_test, X_val, y_train, y_test, y_val = train_test_val_split(X, to_categorical(Y), test_size=0.2)

In [7]:
np.sum(y_train, axis=0)

array([ 826., 1098.], dtype=float32)

In [8]:
X_train, y_train = RandomUnderSampler().fit_resample(X_train, y_train)

C:\Users\sabri\Anaconda3\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass classes=[0 1] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


In [9]:
np.sum(tf.keras.utils.to_categorical(y_train), axis=0)

array([826., 826.], dtype=float32)

In [10]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

C:\Users\sabri\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


RandomForestClassifier(random_state=42)

In [11]:
confusion_matrix(np.argmax(y_test, axis=1),rf.predict(X_test))

array([[ 83,  37],
       [ 57, 111]], dtype=int64)

In [12]:
print(classification_report(np.argmax(y_test, axis=1),rf.predict(X_test)))

              precision    recall  f1-score   support

           0       0.59      0.69      0.64       120
           1       0.75      0.66      0.70       168

    accuracy                           0.67       288
   macro avg       0.67      0.68      0.67       288
weighted avg       0.68      0.67      0.68       288



In [13]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(32, input_shape=(24, 1)))
model.add(tf.keras.layers.Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [14]:
X_train.shape

(1652, 24)

In [15]:
model.fit(reshapeInput(X_train), tf.keras.utils.to_categorical(y_train), 
          epochs=100, batch_size=50,
          validation_data=(reshapeInput(X_val), y_val),
         callbacks=tf.keras.callbacks.EarlyStopping(patience=20, restore_best_weights=True))

Epoch 1/100
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
31/34 [==========================>...] - ETA: 0s - loss: 0.6956 - accuracy: 0.4835WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x00000207FC019828> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`)

34/34 [==============================] - 0s 6ms/step - loss: 0.6101 - accuracy: 0.7044 - val_loss: 0.6262 - val_accuracy: 0.6269
Epoch 46/100
34/34 [==============================] - 0s 6ms/step - loss: 0.6166 - accuracy: 0.6802 - val_loss: 0.5913 - val_accuracy: 0.6995
Epoch 47/100
34/34 [==============================] - 0s 6ms/step - loss: 0.6003 - accuracy: 0.6992 - val_loss: 0.5594 - val_accuracy: 0.7409
Epoch 48/100
34/34 [==============================] - 0s 7ms/step - loss: 0.6024 - accuracy: 0.6904 - val_loss: 0.5663 - val_accuracy: 0.7254
Epoch 49/100
34/34 [==============================] - 0s 7ms/step - loss: 0.6201 - accuracy: 0.6858 - val_loss: 0.5628 - val_accuracy: 0.7409
Epoch 50/100
34/34 [==============================] - 0s 8ms/step - loss: 0.6169 - accuracy: 0.6923 - val_loss: 0.5658 - val_accuracy: 0.6943
Epoch 51/100
34/34 [==============================] - 0s 7ms/step - loss: 0.6013 - accuracy: 0.6978 - val_loss: 0.5871 - val_accuracy: 0.6995
Epoch 52/100
34/34 

In [16]:
confusion_matrix(np.argmax(y_test, axis=1),model.predict_classes(reshapeInput(X_test)))

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


C:\Users\sabri\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


array([[ 72,  48],
       [ 19, 149]], dtype=int64)

In [17]:
print(f"Overall accuracy of ABC Model: {accuracy_score(np.argmax(y_test, axis=1),model.predict_classes(reshapeInput(X_test)))}")

Overall accuracy of ABC Model: 0.7673611111111112


C:\Users\sabri\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Clearly, we can filter the data better but this should suffice for now